In [5]:
import collections as col
import pandas as pd

def count(data):
    uniq_samples = set(data['SampleID'].values)
    uniq_samples |= set(data['FatherID'].values)
    uniq_samples |= set(data['MotherID'].values)
    uniq_samples -= set(['0'])
    return uniq_samples, data.shape[0]

def relationship(a, b):
    child_a, father_a, mother_a = a
    child_b, father_b, mother_b = b
    
    if father_a == father_b and mother_a == mother_b:
        assert child_a != child_b, 'family duplicate: {} v {}'.format(a, b)
        return 'quartet'
    elif (child_a == father_b or child_a == mother_b) or \
            (child_b == father_a or child_b == mother_a):
        return 'multigen'
    else:
        return 'unrelated'

samples_with_data = dict()
with open('PRJEB31736.2504.samples', 'r') as listing:
    for line in listing:
        samples_with_data[line.strip()] = 'PRJEB31736.2504'

with open('PRJEB36890.698.samples', 'r') as listing:
    for line in listing:
        samples_with_data[line.strip()] = 'PRJEB36890.698'

ped_file = '20130606_g1k_3202_samples_ped_population.txt'

df = pd.read_csv(ped_file, sep=' ')

df['is_duo'] = 0
df['is_trio'] = 0
df['is_quartet'] = 0
df['is_multigen'] = 0

duos = ((df['FatherID'] != '0') & (df['MotherID'] == '0')) \
            | ((df['FatherID'] == '0') & (df['MotherID'] != '0'))
df.loc[duos, 'is_duo'] = 1

trios = (df['FatherID'] != '0') & (df['MotherID'] != '0')
df.loc[trios, 'is_trio'] = 1

# check for duplicate family IDs
trio_families = col.defaultdict(list)
for idx, row in df.iterrows():
    if row.FatherID == '0' and row.MotherID == '0':
        continue
    members = row.SampleID, row.FatherID, row.MotherID
    trio_families[getattr(row, '#FamilyID')].append((idx, members))

for fid, family_info in trio_families.items():
    if len(family_info) == 1:
        continue
    if len(family_info) == 3:
        assert family_info[0][1][0] == 'HG03944'
        # one special case for ST116 - incomplete ancestral duo
        select_p = (df['SampleID'] == 'HG03944') & (df['MotherID'] == 'HG04067')
        df.loc[select_p, 'is_multigen'] = 1
        select_f = df['MotherID'] == 'HG03944'
        df.loc[select_f, 'is_multigen'] = 1
        df.loc[select_f, 'is_quartet'] = 1
        continue
    assert len(family_info) == 2, 'nope: {}'.format(family_info)
    if family_info[0][1][1:] == family_info[1][1][1:]:
        # same parents, different children
        # quartet - ok
        df.loc[family_info[0][0], 'is_quartet'] = 1
        df.loc[family_info[1][0], 'is_quartet'] = 1
    else:
        # case same ID, unclear relationship
        rel = relationship(family_info[0][1], family_info[1][1])
        if rel == 'unrelated':
            df.loc[family_info[0][0], '#FamilyID'] += 'A'
            df.loc[family_info[1][0], '#FamilyID'] += 'B'
        elif rel == 'multigen':
            df.loc[family_info[0][0], 'is_multigen'] = 1
            df.loc[family_info[1][0], 'is_multigen'] = 1
        elif rel == 'quartet':
            df.loc[family_info[0][0], 'is_quartet'] = 1
            df.loc[family_info[1][0], 'is_quartet'] = 1
        else:
            raise

multi_family = col.defaultdict(list)
merge_ids = dict()
for idx, row in df.iterrows():
    if row.FatherID == '0' or row.MotherID == '0':
        continue
    members = row.SampleID, row.FatherID, row.MotherID
    multi_family[members[1:]].append((idx, getattr(row, '#FamilyID'), members[0]))

for parents, family_info in multi_family.items():
    if len(family_info) == 1:
        continue
    assert len(family_info) == 2, 'nope2'
    fids = sorted([family_info[0][1], family_info[1][1]])
    if len(set(fids)) == 1:
        continue
    new_fid = 'x'.join(fids)
    for f in fids:
        merge_ids[f] = new_fid

df['#FamilyID'].replace(merge_ids, inplace=True)

# now set is_quartet
for (fid, mid) in [('SH074xSH089', 'HG00657'), ('m008xm011', 'NA19660'), ('m004xm009', 'NA19678')]:
    select = (df['#FamilyID'] == fid) & (df['MotherID'] == mid)
    df.loc[select, 'is_quartet'] = 1

# below - as before, could be simplified now with new indicators...    

df['Sample_Illumina'] = df['SampleID'].apply(lambda x: samples_with_data.get(x, 'n/a'))
df['Father_Illumina'] = df['FatherID'].apply(lambda x: samples_with_data.get(x, 'n/a'))
df['Mother_Illumina'] = df['MotherID'].apply(lambda x: samples_with_data.get(x, 'n/a'))
df['Sex_char'] = df['Sex'].replace({1: 'm', 2: 'f'})
df.sort_values(['Superpopulation', 'Population', '#FamilyID', 'SampleID'], inplace=True)

samples, _ = count(df)
ped_aug = '20130606_g1k_{}_samples_ped_population.ext.tsv'.format(len(samples))
df.to_csv(ped_aug, sep='\t', header=True, index=False)

unsampled_individuals = set(df.loc[df['Sample_Illumina'] == 'n/a', 'SampleID'].values)
unsampled_individuals |= set(df.loc[df['Father_Illumina'] == 'n/a', 'FatherID'].values)
unsampled_individuals |= set(df.loc[df['Mother_Illumina'] == 'n/a', 'MotherID'].values)
unsampled_individuals -= set(['0'])

unsampled_file = '20130606_g1k_{}_samples_no-data.txt'.format(len(unsampled_individuals))
with open(unsampled_file, 'w') as dump:
    dump.write('\n'.join(sorted(unsampled_individuals)) + '\n')

sampled_individuals, _ = count(df)
sampled_individuals -= unsampled_individuals
sampled_file = '20130606_g1k_{}_samples_with-data.txt'.format(len(sampled_individuals))
with open(sampled_file, 'w') as dump:
    dump.write('\n'.join(sorted(sampled_individuals)) + '\n')

has_mother = df['Mother_Illumina'] != 'n/a'
has_father = df['Father_Illumina'] != 'n/a'
is_family = has_mother & has_father

trios_with_data = df.loc[is_family, :].copy()
samples, trios = count(trios_with_data)
ped_trios = '20130606_g1k_{}-samples_{}-trios_ped.families.tsv'.format(len(samples), trios)
trios_with_data.to_csv(ped_trios, sep='\t', header=True, index=False)

ped_trios = '20130606_g1k_{}-samples_{}-trios.families.txt'.format(len(samples), trios)
with open(ped_trios, 'w') as dump:
    dump.write('\n'.join(sorted(samples)) + '\n')


# father_dup = trios_with_data.duplicated('FatherID', keep=False)
# mother_dup = trios_with_data.duplicated('MotherID', keep=False)
# siblings = trios_with_data.loc[father_dup | mother_dup, :].copy()
# samples, trios = count(siblings)
# ped_siblings = '20130606_g1k_{}-samples_{}-trios_ped.siblings.tsv'.format(len(samples), trios)
# siblings.to_csv(ped_siblings, sep='\t', header=True, index=False)

# ped_siblings = '20130606_g1k_{}-samples_{}-trios.siblings.txt'.format(len(samples), trios)
# with open(ped_siblings, 'w') as dump:
#     dump.write('\n'.join(sorted(samples)) + '\n')


#### dump quartets for phasing
quartets = df.loc[df['is_quartet'] == 1, :].copy()
samples, trios = count(quartets)
ped_quartets = '20130606_g1k_{}-samples_{}-trios_ped.quartets.tsv'.format(len(samples), trios)
quartets.to_csv(ped_quartets, sep='\t', header=True, index=False)

ped_quartets = '20130606_g1k_{}-samples_{}-trios.quartets.txt'.format(len(samples), trios)
with open(ped_quartets, 'w') as dump:
    dump.write('\n'.join(sorted(samples)) + '\n')


#### dump multi-generational
multigen = df.loc[df['is_multigen'] == 1, :].copy()
# drop those with only data for the child
has_father = df['Father_Illumina'] != 'n/a'
has_mother = df['Mother_Illumina'] != 'n/a'
multigen = multigen.loc[has_father | has_mother, :].copy()
# now make sure a family ID exists at least twice
multigen['count'] = multigen['#FamilyID'].apply(lambda x: multigen['#FamilyID'].value_counts()[x])
multigen = multigen.loc[multigen['count'] > 1, :].copy()
multigen.drop('count', axis=1, inplace=True)

samples, trios = count(multigen)
ped_multigen = '20130606_g1k_{}-samples_{}-trios_ped.multigen.tsv'.format(len(samples), trios)
multigen.to_csv(ped_multigen, sep='\t', header=True, index=False)

ped_multigen = '20130606_g1k_{}-samples_{}-trios.multigen.txt'.format(len(samples), trios)
with open(ped_multigen, 'w') as dump:
    dump.write('\n'.join(sorted(samples)) + '\n')


#### dump: step siblings (father unknown)
step_siblings = df.loc[df['MotherID'] == 'NA19238', :].copy()
samples, trios = count(step_siblings)
ped_stepsiblings = '20130606_g1k_{}-samples_{}-trios_ped.step-siblings.tsv'.format(len(samples), trios)
step_siblings.to_csv(ped_stepsiblings, sep='\t', header=True, index=False)

ped_stepsiblings = '20130606_g1k_{}-samples_{}-trios.step-siblings.txt'.format(len(samples), trios)
with open(ped_stepsiblings, 'w') as dump:
    dump.write('\n'.join(sorted(samples)) + '\n')

#### dump: individuals
individuals = df.loc[(df['FatherID'] == '0') & (df['MotherID'] == '0'), :].copy()
samples, _ = count(individuals)
ped_individuals = '20130606_g1k_{}-samples_0-trios_ped.individuals.tsv'.format(len(samples))
individuals.to_csv(ped_individuals, sep='\t', header=True, index=False)

ped_individuals = '20130606_g1k_{}-samples_0-trios.individuals.txt'.format(len(samples))
with open(ped_individuals, 'w') as dump:
    dump.write('\n'.join(sorted(samples)) + '\n')

### dump singletons
individuals['count'] = individuals['#FamilyID'].apply(lambda x: individuals['#FamilyID'].value_counts()[x])
singletons = individuals.loc[individuals['count'] == 1, :].copy()
singletons.drop('count', axis=1, inplace=True)
samples, _ = count(singletons)
ped_singletons = ped_individuals = '20130606_g1k_{}-samples_0-trios_ped.singletons.tsv'.format(len(samples))
singletons.to_csv(ped_singletons, sep='\t', header=True, index=False)

ped_singletons = '20130606_g1k_{}-samples_0-trios.singletons.txt'.format(len(samples))
with open(ped_singletons, 'w') as dump:
    dump.write('\n'.join(sorted(samples)) + '\n')
    
#### special: pilot samples, 100 trios
pilot_samples = open('samples-pilot.tsv', 'r').read().strip().split()

pilot_child = trios_with_data['SampleID'].isin(pilot_samples)
pilot_fathers = trios_with_data['FatherID'].isin(pilot_samples)
pilot_mothers = trios_with_data['MotherID'].isin(pilot_samples)
pilot_families = pilot_child & pilot_fathers & pilot_mothers

pilot_trios = trios_with_data.loc[pilot_families, :].copy()
samples, trios = count(pilot_trios)

pilot_file = '20130606_g1k_{}-samples_{}-trios_ped.pilot.tsv'.format(len(samples), trios)
pilot_trios.to_csv(pilot_file, sep='\t', header=True, index=False)

pilot_file = '20130606_g1k_{}-samples_{}-trios.pilot.txt'.format(len(samples), trios)
with open(pilot_file, 'w') as dump:
    dump.write('\n'.join(sorted(samples)) + '\n')